# OANDA_RESTAPI_FX_Logic explanation

<p>■AI(機械学習）のxgboostとは<br>
こちらが詳しいです。<br>
<a href="http://tjo.hatenablog.com/entry/2015/05/15/190000" class="autolink" rel="nofollow noopener" target="_blank">http://tjo.hatenablog.com/entry/2015/05/15/190000</a></p>

<p>要は、入力したデータ群から、「何か」を分類できる分類器ですね。<br>
最近流行りの機械学習のライブラリです。<br>
分類のルールを教師データから学習することができます。<br>
なお、内部のアルゴリズムは決定木学習？をブースティング？でやっているらしいです。<br>


<p>■システムトレードの考え<br>
テクニカル指標群+αを特徴量として入力し、回帰にって為替が上昇・下降するかの2値分類を行う。<br>
この結果を主軸に、工夫してトレーディングを行っていく。</p>

<p>過去の一定期間のチャートの情報を集約したテクニカル指標を見てやれば、チャートの動きも読めるのでは、と考えたのが経緯です。</p>

<p>■プログラムの流れ<br>
1. 学習<br>
過去の為替データからテクニカル指標群+αと入力期間から3足後が上昇するか下降するかを求めて、そのペアを作ってxgboostに学習させる。<br>
論文で、上下を反転させたチャートについても学習するとパフォーマンスが良くなると書いてあり、そういうこともしています。</p>

<p>入力しているテクニカル指標群については下の記事で整理しています。<br>
+αは、前日との差分や当日までのチャートのタイプです。</p>

<p>"シストレプログラムで特徴量として利用しているテクニカル指標を整理しているものです。"<br>
<a href="https://www.dropbox.com/s/5sspkpyx5c7vd5z
" class="autolink" id="reference-92df7ac6f3fdd43bf53d">https://www.dropbox.com/s/5sspkpyx5c7vd5z
</a></p>


<p>2.予測及びポジションの購入<br>
学習の時と同じようにして入力データを作成し、予測を行います。予測結果はスカラ値として得られ、1に近いほど上昇する可能性が高く、0にちかいほど下降する可能性が高いことを示します。<br>
この結果が閾値(High)より高ければ買いポジションを、閾値(Low)より低ければ売りポジションを持ちます。<br>
これが基本的な考え方ですが、他にもボラリティが高い場合は見送る、現時点までのチャートタイプが大体単調での減少か、増加ではないと見送るといった条件が加えられています。買いポジションを持つ場合は過去のチャートが減少タイプでないとならず、売りはその逆といった条件になっているので、このロジックは逆張りを狙った動作をするものだと言えます。<br>
なお、ポジションは全資産を用いて購入し(レバレッジ1倍)ます。このため、上記プログラムは複利で資産が変化します。<br>
また、ある時点でポジションは最大1つしか持ちません。</p>

<p>3.決済<br>
決済は購入後常に試みます。その際にチャートタイプが大体単調増加・減少であれば決済します。この際、勝ち負けは見ておらず、どちらであっても決済してしまいます。</p>

<p>■バックテストの結果（参考）<br>
USDJPYで、スプレッド0.3銭、レバレッジ1倍、初期資金100万円で上記プログラムを動作させた結果が、資産推移は以下のようになりました。学習期間が2001年～2002年冒頭、バックテスト期間がそれから2008年末までです。データは5分足です。</p>

<p>ベースライン（同じタイミングで全て買いポジを購入した場合）<br>
<a href="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F7b623aec-6b9a-2797-b939-58e50d4f19ef.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;s=9e77c39da40dfdf44a8abda9d39e11d6" target="_blank" rel="nofollow noopener"><img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F7b623aec-6b9a-2797-b939-58e50d4f19ef.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;s=9e77c39da40dfdf44a8abda9d39e11d6" alt="all_long.png" data-canonical-src="https://qiita-image-store.s3.amazonaws.com/0/12325/7b623aec-6b9a-2797-b939-58e50d4f19ef.png" srcset="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F7b623aec-6b9a-2797-b939-58e50d4f19ef.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;w=1400&amp;fit=max&amp;s=832077af503897472926899158dc7064 1x" loading="lazy"></a></p>

<p>ベースライン（同じタイミングで全て売りポジを購入した場合）<br>
<a href="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F0323445b-c438-31a8-673f-d75f619b2556.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;s=e06709c4ff90ff8b0d792ec5ad0b4f35" target="_blank" rel="nofollow noopener"><img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F0323445b-c438-31a8-673f-d75f619b2556.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;s=e06709c4ff90ff8b0d792ec5ad0b4f35" alt="all_short.png" data-canonical-src="https://qiita-image-store.s3.amazonaws.com/0/12325/0323445b-c438-31a8-673f-d75f619b2556.png" srcset="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F0323445b-c438-31a8-673f-d75f619b2556.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;w=1400&amp;fit=max&amp;s=093c3d745b64f531bc662d2f799416c4 1x" loading="lazy"></a></p>

<p>xgboostで予測した方のポジを購入した場合<br>
<a href="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F493a4900-d274-3c59-5b0f-4a75a2ccdc53.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;s=688aacc402106c6e89cfa222a9b7cff3" target="_blank" rel="nofollow noopener"><img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F493a4900-d274-3c59-5b0f-4a75a2ccdc53.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;s=688aacc402106c6e89cfa222a9b7cff3" alt="1093c81c770524e53e0ee4acc883d05e.png" data-canonical-src="https://qiita-image-store.s3.amazonaws.com/0/12325/493a4900-d274-3c59-5b0f-4a75a2ccdc53.png" srcset="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F12325%2F493a4900-d274-3c59-5b0f-4a75a2ccdc53.png?ixlib=rb-1.2.2&amp;auto=compress%2Cformat&amp;gif-q=60&amp;w=1400&amp;fit=max&amp;s=3cbba7a9c2c77cdaa872013294a90b6a 1x" loading="lazy"></a></p>

